In [ ]:
import codecs
import csv
import os
from tkinter import filedialog
import glob
import pandas as pd
import pprint

from pip._vendor import chardet
from detect_delimiter import detect

from HakatonScripts.MyFunctions import sort_df

In [ ]:
def select_folders():
    input_data_path = filedialog.askdirectory(title="Выберите папку с необработанными файлами")
    output_data_path = filedialog.askdirectory(title="Выберите папку для обработанных файлов")

    # Проверяем что пути к файлам указаны
    if (len(input_data_path) == 0 or len(output_data_path) == 0):
        print("Папки не выбраны. Перезапустите программу и выберете файлы!")
        raise SystemExit

    return input_data_path, output_data_path

def open_folder(folder_path: str):
    os.startfile(folder_path)

def get_files_in_folder(folder_path:str):
    all_files = glob.glob(os.path.join(folder_path, "*.csv"))  # advisable to use os.path.join as this makes concatenation OS independent
    return all_files

def get_dataframe_from_file(file_path:str)  -> pd.DataFrame:
    df_from__file = (pd.read_csv(file_path))
    return df_from__file

def get_dataframes_from_files_list(files_list, table_names_list):
    dataframes = []
    for table_name in table_names_list:
        for file in files_list:
            if(table_name in os.path.basename(file).split()[0]):
                #has_header = check_header_exist(file)
                #if (has_header):
                if(table_name == 'sales'): # hardcode
                    df = pd.read_csv(file, encoding=get_encoding(file), sep=get_separator(file), header=0)
                else:
                    df = pd.read_csv(file, encoding=get_encoding(file), sep=get_separator(file), header=None)

                #df = pd.read_csv(file, encoding=get_encoding(file), sep='\t', header=csv.Sniffer().has_header(file.read(1024)))
                dataframes.append(df)
    return dataframes

def get_encoding(file_path:str):
    with open(file_path, 'rb') as rawdata:
        result = chardet.detect(rawdata.read(100))
    return result['encoding']

def get_separator(file_path:str): # https://stackoverflow.com/questions/46135839/auto-detect-the-delimiter-in-a-csv-file-using-pd-read-csv
    with codecs.open(file_path, "r", get_encoding(file_path)) as f:
        firstline = f.readline()
        separator = detect(firstline)
        #print(f'{file_path} separator: {separator}')
    return separator

'''
def check_header_exist(file_path:str):  # https://stackoverflow.com/questions/40193388/how-to-check-if-a-csv-has-a-header-using-python
    def _get_data(self, filename):
        sniffer = csv.Sniffer()
        training_data = ''
        with open(filename, 'rt') as csvfile:
            dialect = csv.Sniffer().sniff(csvfile.read(2048))
            training_data = csv.reader(csvfile, delimiter=dialect.delimiter, encoding=get_encoding(filename))
            csvfile.seek(0)
            has_header = ''.join(next(training_data)).isalpha()
            print(f'filename has header: {has_header}')
            csvfile.seek(0)
            return has_header
'''



In [ ]:
if __name__ == '__main__':
    input_data_path_variable, output_data_path_variable = select_folders()
    input_data_files = get_files_in_folder(input_data_path_variable)



    #sales_df, base_info_client_df, okved_industry_client_df, address_client_df = get_dataframes_from_files_list(input_data_files, ['sales', 'base_info_client', 'okved_industry_client', 'address_client'])
    sales_df, base_info_client_df, okved_industry_client_df = get_dataframes_from_files_list(input_data_files, ['sales', 'base_info_client', 'okved_industry_client'])
    sales_df.to_csv(output_data_path_variable + '/sales.csv')

    print(sales_df.info)
    print(f'Исходное количество единиц наблюдения: {sales_df.shape[0]}')
    print(f'Количество пропусков по столбцам: {sales_df.sum()}')
    print(f'Количество явных дубликатов: {sales_df.duplicated().sum()}')
    #sales_df.hist(figsize=(15, 20));

    #Удаляю дубликаты в sales_df с уважением к категории покупки
    sales_df['SALE_SIZE'] = pd.Categorical(sales_df['SALE_SIZE'], categories=['High', 'Middle', 'Low'], ordered=True)
    sales_df.sort_values('SALE_SIZE', ascending=False).drop_duplicates(['ID_CLIENT_RF'])

    #sales_df.sort_values(['ID_CLIENT_RF', 'SALE_SIZE']).drop_duplicates('ID_CLIENT_RF', keep="last")
    sales_df.to_csv(output_data_path_variable + '/sales.csv', sep= ',')

    #print(sales_df.head())
    #print(base_info_client_df.head())
    #print(okved_industry_client_df.head())
    #print(address_client_df.head())

     #sales_df.describe()

    #base_info_client_df.info()
    #okved_industry_client_df.info()
    #address_client_df.info()

    #open_folder(input_data_path_variable)
    #open_folder(output_data_path_variable)